In [1]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import xmltodict
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import os
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain.chains import RetrievalQAWithSourcesChain,VectorDBQAWithSourcesChain,ConversationalRetrievalChain

In [3]:
def extract_text_from(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)

In [4]:
# r = requests.get("https://sbnri.com/sbnri-blog-sitemap.xml")
# xml = r.text
# raw = xmltodict.parse(xml)

# site_urls = []

# for item in raw['sitemapindex']['sitemap']:
#     site_urls.append(item['loc'])

# site_urls

['https://sbnri.com/sbnri-blog-definitions-sitemap.xml',
 'https://sbnri.com/sbnri-blog-education-sitemap.xml',
 'https://sbnri.com/sbnri-blog-entertainment-and-lifestyle-2-sitemap.xml',
 'https://sbnri.com/sbnri-blog-news-room-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-bank-accounts-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-fixed-deposit-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-income-tax-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-insurance-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-investment-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-loans-sitemap.xml',
 'https://sbnri.com/sbnri-blog-nri-mutual-fund-sitemap.xml',
 'https://sbnri.com/sbnri-blog-passport-sitemap.xml',
 'https://sbnri.com/sbnri-blog-real-estate-sitemap.xml',
 'https://sbnri.com/sbnri-blog-remittance-sitemap.xml',
 'https://sbnri.com/sbnri-blog-repatriation-sitemap.xml',
 'https://sbnri.com/sbnri-blog-retirement-sitemap.xml',
 'https://sbnri.com/sbnri-blog-travel-sitemap.xml',
 'ht

In [5]:
# pages = []

# for site_url in site_urls:
#     try:
#         r = requests.get(site_url)
#         xml = r.text
#         raw = xmltodict.parse(xml)
#         print(raw)
#         for info in raw['urlset']['url']:
#             url = info['loc']
#             pages.append({'text': extract_text_from(url), 'source': url})
#     except:
#         print("Error")

Error
{'urlset': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:image': 'http://www.google.com/schemas/sitemap-image/1.1', '@xsi:schemaLocation': 'http://www.sitemaps.org/schemas/sitemap/0.9 http://www.sitemaps.org/schemas/sitemap/0.9/sitemap.xsd http://www.google.com/schemas/sitemap-image/1.1 http://www.google.com/schemas/sitemap-image/1.1/sitemap-image.xsd', '@xmlns': 'http://www.sitemaps.org/schemas/sitemap/0.9', 'url': [{'loc': 'https://sbnri.com/blog/education/part-time-job-options-in-saudi-arabia', 'lastmod': '2022-08-19T13:31:37+00:00', 'changefreq': 'daily', 'priority': '0.8'}, {'loc': 'https://sbnri.com/blog/education/7-ways-to-protect-your-workplace-from-hackers', 'lastmod': '2022-09-24T11:20:11+00:00', 'changefreq': 'daily', 'priority': '0.8'}, {'loc': 'https://sbnri.com/blog/education/best-job-options-for-indians-in-italy', 'lastmod': '2022-10-03T19:23:48+00:00', 'changefreq': 'daily', 'priority': '0.8'}, {'loc': 'https://sbnri.com/blog/education/8-top-

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],chunk_overlap=200,)
# doc_chunks = []
# for page in pages:
#     chunks = text_splitter.split_text(page['text'])
#     for i, chunk in enumerate(chunks):
#             doc = Document(
#                 page_content=chunk,
#                 metadata={"source": page['source']},
#             )
#             doc_chunks.append(doc)
#     print(f"Split {page['source']} into {len(chunks)} chunks")

Split https://sbnri.com/blog/education/part-time-job-options-in-saudi-arabia into 21 chunks
Split https://sbnri.com/blog/education/7-ways-to-protect-your-workplace-from-hackers into 23 chunks
Split https://sbnri.com/blog/education/best-job-options-for-indians-in-italy into 22 chunks
Split https://sbnri.com/blog/education/8-top-countries-offering-scholarships-for-indians into 27 chunks
Split https://sbnri.com/blog/education/tips-for-every-parent-to-save-their-child-from-sexual-abuse into 23 chunks
Split https://sbnri.com/blog/education/12-best-job-options-for-indian-students-in-uk into 21 chunks
Split https://sbnri.com/blog/education/gre-subject-test-registration-process into 21 chunks
Split https://sbnri.com/blog/education/top-10-scams-in-world-and-how-to-prevent-them into 24 chunks
Split https://sbnri.com/blog/education/freeway-vs-highway-whats-the-difference into 25 chunks
Split https://sbnri.com/blog/education/indians-are-the-second-largest-group-of-international-students-in-germany

In [ ]:
embeddings = OpenAIEmbeddings()

# vector_store = Chroma.from_documents(
#     doc_chunks,
#     embeddings,
#     collection_name="sbnri",
#     persist_directory="src/data/chroma",
# )

# # Save DB locally
# vector_store.persist()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
def get_open_ai_model(model_name,temperature = 0, openai_api_key = os.getenv("OPENAI_API_KEY"),chat_model = True, verbose = False):
    if chat_model:
        return ChatOpenAI(model_name=model_name,temperature=temperature,openai_api_key=openai_api_key,verbose=verbose)
    else:
        return OpenAI(model_name=model_name,temperature=temperature,openai_api_key=openai_api_key,verbose=verbose)
    
def get_vectorstore(collection_name,persist_dir,embedding = None):
    if embedding is None:
        embedding = OpenAIEmbeddings()

    vector_store = Chroma(
        collection_name=collection_name,
        embedding_function=embedding,
        persist_directory=persist_dir,
    )

    return vector_store

def make_conversational_chain(model, vector_store ,memory = None , return_source_documents = True , verbose = False, system_prompt = None):
    if memory is None:    
        memory = ConversationSummaryBufferMemory(llm=model , memory_key="chat_history", return_messages=True,input_key='question', output_key='answer')

    if system_prompt is None:
        chain = ConversationalRetrievalChain.from_llm(
        model, 
        vector_store.as_retriever(), 
        memory=memory,
        return_source_documents=return_source_documents,
        verbose=verbose
        )
        return chain

    chain = ConversationalRetrievalChain.from_llm(
    model, 
    vector_store.as_retriever(), 
    memory=memory,
    return_source_documents=return_source_documents,
    combine_docs_chain_kwargs=dict(prompt=system_prompt),
    verbose=verbose
    )

    return chain

In [6]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    model = get_open_ai_model("gpt-3.5-turbo",verbose= True,temperature=0.3)
    vector_store = get_vectorstore("sbnri","src/data/chroma")

    system_prompt = PromptTemplate.from_template('''
    You are a customer support guide representing SBNRI, a reputable online platform known for solving the banking needs of NRI's in India. 
    Your role is to assist customers by providing accurate information, offering helpful recommendations, and guiding them towards the solutions of their issues. 
    Feel free to ask clarifying questions to better understand the customer's needs and preferences. Leverage the provided context to answer the question effectively 
    without generating false or fictional information. Double check your response for accuracy. Your responses should be short and friendly.
    Respond only to the following question using only the context and if you don't know the answer respond with "May I connect you with an expert in this topic to discuss this in detail?":

    Context: {context}
    Question: {question}

    You can also ask questions to gain more insights and lead the customer more accurately. 
    Remember, your expertise and helpfulness are key in assisting customers in making informed choices.'''
    )


    # strict_system_prompt = PromptTemplate.from_template('''
    # Respond to the following question using only the context, and if you don't know the answer, 
    # simply respond accordingly without making up information:

    # Context: {context}
    # Question: {question}

    # You can also ask questions to gather more insights and guide the user more effectively. Remember, your expertise and helpfulness are crucial in assisting users 
    # with their technical concerns and providing them with the best solutions.'''
    # )


    chain = make_conversational_chain(model,vector_store,system_prompt=system_prompt)
    
    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        response = chain({"question": history[-1][0]})
        source = response["source_documents"]
        urls = ""
        print("\n\nSources:\n")
        for document in source:
            print(f"Url: {document.metadata['source']}")
            urls += document.metadata['source']
            urls += "\n"
        bot_message = response["answer"] + "\n" + urls
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.01)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: chain.memory.clear() ,  None, chatbot, queue=False)


In [7]:
demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0c89a50c26557c7289.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces




Sources:

Url: https://sbnri.com/blog/nri-loans/nri-car-loan-in-india
Url: https://sbnri.com/blog/nri-loans/best-banks-for-nri-loan-against-property
Url: https://sbnri.com/blog/nri-mutual-fund/net-asset-value-or-nav
Url: https://sbnri.com/blog/nri-mutual-fund/best-mutual-funds-in-india


Sources:

Url: https://sbnri.com/blog/nri-loans/nri-car-loan-in-india
Url: https://sbnri.com/blog/nri-loans/best-banks-for-nri-loan-against-property
Url: https://sbnri.com/blog/nri-mutual-fund/net-asset-value-or-nav
Url: https://sbnri.com/blog/nri-mutual-fund/best-mutual-funds-in-india


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 4d8888670d5e9fd71a4b7cbe3d03bd95 in your message.).




Sources:

Url: https://sbnri.com/blog/retirement/admission-process-in-cbse-schools-for-nris-moving-to-india
Url: https://sbnri.com/blog/retirement/admission-process-in-cbse-schools-for-nris-moving-to-india
Url: https://sbnri.com/blog/travel/wardrobe-shopping-list-for-indians-moving-to-the-us
Url: https://sbnri.com/blog/news-room/in-talk-with-the-hindu-mudit-vijayvergiya-explains-how-sbnri-is-helping-nris-with-banking-and-investment-in-india


Sources:

Url: https://sbnri.com/blog/education/top-10-scams-in-world-and-how-to-prevent-them
Url: https://sbnri.com/blog/education/gre-subject-test-registration-process
Url: https://sbnri.com/blog/education/tips-for-every-parent-to-save-their-child-from-sexual-abuse
Url: https://sbnri.com/blog/education/gre-subject-test-registration-process


Sources:

Url: https://sbnri.com/blog/nri-loans/nri-car-loan-in-india
Url: https://sbnri.com/blog/nri-loans/best-banks-for-nri-loan-against-property
Url: https://sbnri.com/blog/nri-mutual-fund/net-asset-va